In [75]:
#import config  # Import your config.py file this contains you openai api key
import pandas as pd
import numpy as np
import os
from llm_comparison_toolkit import RateLimiter, get_response_openai, get_response_anthropic,  create_config_dict_func, use_df_to_call_llm_api, compare_request_configurations


#load the dev and test sets for prompt development and selection
dev_data_df = pd.read_csv('data/dev_data_raw.csv')
test_data_df = pd.read_csv('data/test_data_raw.csv')

# Explore different system prompts

This explores a range of system prompt to find the one that appears to work the best, we use gpt4 as the baseline model

In [77]:
#Create a modular set of system messages that can be combined in different ways
basic_prompt = "Please recover the text from the corrupted OCR"
expertise_prompt = "You are an expert in post-OCR correction of documents."
recover_prompt = "Using the context available from the text please recover the most likely original text from the corrupted OCR."
publication_context_prompt = "The text is from an english newspaper in the 1800's"
text_context_prompt = "The text may be an advert or article and may be missing the beggining or end"
additional_instructions_prompt = "Do not add any text, commentary, or lead in sentences  beyond the recovered text. Do not add a title, or any introductions"

#combine all the message parts into a variety of system messages
#N.B. This is not and exhaustive combination as that would be very expensive and likley not yield significantly better results
system_messages_list = [
('basic_prompt', basic_prompt),
('expert_basic_prompt', expertise_prompt + ' '+ basic_prompt),
('expert_recover_prompt', expertise_prompt + ' '+ recover_prompt),
('expert_recover_publication_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + publication_context_prompt),
('expert_recover_text_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + text_context_prompt),
('expert_recover_publication_text_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + publication_context_prompt + ' ' + text_context_prompt),
('expert_recover_instructions_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + additional_instructions_prompt),
('full_context_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + publication_context_prompt + ' ' + text_context_prompt + additional_instructions_prompt)
]


#Loop through all the system messages and generate a config dictionary for each one
message_test_configs = []

for iter_system_message in system_messages_list:

    message_test_configs.append(
        
        create_config_dict_func(
        get_response_func = get_response_openai,
        rate_limiter = RateLimiter(50000),
        engine = "gpt-3.5-turbo",
        system_message_template = iter_system_message[1],
        prompt_template =  "{content_html}",
        additional_args={'response_name':iter_system_message[0]}
    )
    )

In [78]:
#Run on both the dev and test sets.

compare_request_configurations(dev_data_df, message_test_configs, folder_path='./data/dev_system_message_variants')

compare_request_configurations(test_data_df, message_test_configs, folder_path='./data/test_system_message_variants')

# Create basic and no message responses

Having identified the optimal system message we can now compare the different LLM's. In this experiment it is assumed that the prompt (which is the system message) does not need to be changed but that it may not work as a system message and may need to placed AFTER the text in the prompt message itself.

The below code creates the basic configuration dictionaries for each model and then fills in the with the two different prompt messages creating a single list of all basic prompt/message/model configurations. It then calls all the LLM's and saves the results.
This works in series so takes a while.

In [71]:
#Create the prompt/system message using the best performing from the previous section

system_message = f"""Please recover the text from the corrupted OCR, providing appropriate paragraph breaks. 
Do not add any text, commentary, or lead in sentences  beyond the recovered text. Do not add a title, or any introductions"""

no_prompt = no_message = ""

prompt_content_only = "{content_html}"

prompt_content_message =  "{content_html}"+f""" \n \n """+ system_message

In [ ]:
groq_alt_endpoint = {'alt_endpoint':{'base_url':'https://api.groq.com/openai/v1',
                     'api_key':os.getenv("GROQ_API_KEY")}}

basic_model_configs = pd.DataFrame({
    'get_response_func': [get_response_openai, get_response_openai, get_response_anthropic, get_response_anthropic, 
                          get_response_openai, get_response_openai, get_response_openai], 
    'engine': ['gpt-3.5-turbo', 'gpt-4-turbo-preview', "claude-3-haiku-20240307", "claude-3-opus-20240229", 
               'mixtral-8x7b-32768', 'llama2-70b-4096', 'gemma-7b-it'],
    'additional_args': [
        {}, {}, {}, {}, 
        groq_alt_endpoint, 
        groq_alt_endpoint, 
        groq_alt_endpoint
    ]
})

base_model_configs= []

for index, row in basic_model_configs.iterrows():
    #modify the response name for the type
    row['additional_args']['response_name'] = 'base'
    base_model_configs.append(

        create_config_dict_func(
    get_response_func = row['get_response_func'],
    rate_limiter = RateLimiter(40000),
    engine = row['engine'],
    system_message_template = system_message,
    prompt_template =  prompt_content_only,
    additional_args=row['additional_args']
    )

    )

nosm_model_configs = []

for index, row in basic_model_configs.iterrows():
    #modify the response name for the type
    row['additional_args']['response_name'] = 'nosm' 
    nosm_model_configs.append(

        create_config_dict_func(
    get_response_func = row['get_response_func'],
    rate_limiter = RateLimiter(40000),
    engine = row['engine'],
    system_message_template = no_message,
    prompt_template =  prompt_content_message,
    additional_args=row['additional_args']
    )

    )


model_configs = base_model_configs + nosm_model_configs

compare_request_configurations(dev_data_df, model_configs, folder_path='./data/dev_corrected_base')
    

In [ ]:
system_message = f"""Please recover the text from the corrupted OCR, providing appropriate paragraph breaks. 
Do not add any text, commentary, or lead in sentences  beyond the recovered text. Do not add a title, or any introductions"""

system_message = f"""You are an expert at post-OCR correction of documents. Please recover the most likely original text from the corrupted OCR taken from an english newspaper in the 1800's, 
Please provide appropriate line and paragraph breaks. 
Do not add any text, commentary, or lead in sentences  beyond the recovered text. Do not add a title, or any introductions"""

system_message = f"""You are an expert at post-OCR correction of documents. Please recover the most likely original text from the corrupted OCR taken from an english newspaper in the 1800's, 
Please provide appropriate line and paragraph breaks. 
Do not add any text, commentary, or lead in sentences  beyond the recovered text. Do not add a title, or any introductions"""


In [3]:
OCR_text = f"""fcm E ¥ gNlGP «® WOTH ) THE R&PP ^ Ol SPBRBTTS . [ The following Letter was written at our request . The writer , a German friend , whose integrity and clear-headedness would command attention to whatever he might assert , informed us of his proposed visit to a house where the Rappitcs were to exhibit , and we begged him to furnish us with a report . We print his letter without any alteration : its statements will , we think , set the reader speculating .Let it be distinctly borne in mind , however , that in printing this Letter we give no countenance whatever to the pretended explanation of " Rapping . " We have our own views of the jugglery by which the "ghosts " are made to communicate with persons willing to pay for the interview , and in a very short time we hope to lay before our readers something like an experimental result . ] My dear Sir , —When I proposed to give you a report on these new prophets , who , if as true as they arc new , open to us a wide and most interesting field for inquiry , overthrow ancient and modem systems of science and belief , shake to the very foundation revealed religion andChristianity , hut , on the other hand , arc telling almost equally strong against Pantheism mid Atheism , I had not seen those prophets , expounders , mediums , or whatever you may call them , myself ; hut what I heard from a friend , a clear-sighted , well-informed , by no moans " gullible , " or over-credulous gentleman , who had paid them ii visit , had made me anxious to see and judge for myself ; and he having determined , for better satisfaction , to have the Medium ( and the spirits ) at his own house , and having kindly invited me to be present on the occasion , 1 offered to furnish you with a statement of the result of the evening , which I now , agreeably to your wish , lay before you .We were five of us in the library , —my friend , his wife , his sister , his nephew , and myself , —when the footman , handing in a card , announced "Mrs . JIayden . " Her entrance and deportment were easy , unembarrassed , and yet not " business-like ; " her exterior rather prepossessing ; an intelligent countenance , with , perhaps , a slight touch of Yankccisin in the corner of the eye ; and the conversation soon bring established , showed that she did not lack those powers of speech so peculiar to the . citizens of the great Republic ; though certain mistakes now and then recurring in the conjugation of verbs , indicated that nhe could not have had a very fin ;! -rate education . We took our seats round the table on which the card had been placed , [ mid , — "Mr . and Mrs . W . R . Jlirydeii , L' 2 , Queen Anne-street , Cavenuish-. squarc . " Expectation created silence , now and then broken by . qiiestions in a low voice , addressed to the Medium , and by the wheels of carriages that brought guests to an evening party at a neighbouring house , and thus tnadd the otherwise very quirt sired ; liither noisy . Tho raps which the Medium and one or two of us , after we had been seated for about ten"""

In [4]:
system_message = f"""Please recover the text from the corrupted OCR, providing appropriate paragraph breaks. 
Do not add any text, commentary, or lead in sentences  beyond the recovered text. Do not add a title, or any introductions"""



prompt = OCR_text + "\n\n" + f"""Please recover the text from the corrupted OCR, providing appropriate paragraph breaks. Do not add any text, commentary, or lead in sentences  beyond the recovered text"""

#system_message = ""

#prompt = OCR_text

In [6]:
configurations = [create_config_dict_func(
    get_response_func = get_response_anthropic,
    rate_limiter = RateLimiter(40000),
    engine = "claude-3-haiku-20240307",
    system_message_template = "",
    prompt_template =  "{content_html}"+f""" \n \n """+ system_message,
    additional_args={'response_name':'haiku_nosm_base'}
),

create_config_dict_func(
    get_response_func = get_response_anthropic,
    rate_limiter = RateLimiter(40000),
    engine = "claude-3-opus-20240229",
    system_message_template = system_message,
    prompt_template =  "{content_html}",
    additional_args={'response_name':'opus_base'}
)]





In [8]:
use_df_to_call_llm_api(config_dict, dev_data_raw_df, 'haiku_base' , folder_path='./data/dev_corrected')

/home/jonno/redigitalize/llm_caller.py:237: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  times_df = pd.concat([times_df, pd.DataFrame([new_records], index = [0])], ignore_index=True)


In [ ]:
rate_limiter = RateLimiter(50000)

test = get_response_anthropic(prompt, system_message, rate_limiter, engine="claude-3-haiku-20240307", max_tokens = 4000)

In [ ]:
message

Message(id='msg_01RDuf2hzxmMSCQHRQkQT9XH', content=[ContentBlock(text='Here is the text recovered from the corrupted OCR, with appropriate paragraph breaks:\n\nMy dear Sir, —When I proposed to give you a report on these new prophets, who, if as true as they are new, open to us a wide and most interesting field for inquiry, overthrow ancient and modern systems of science and belief, shake to the very foundation revealed religion and Christianity, but, on the other hand, are telling almost equally strong against Pantheism and Atheism, I had not seen those prophets, expounders, mediums, or whatever you may call them, myself; but what I heard from a friend, a clear-sighted, well-informed, by no means "gullible," or over-credulous gentleman, who had paid them a visit, had made me anxious to see and judge for myself; and he having determined, for better satisfaction, to have the Medium (and the spirits) at his own house, and having kindly invited me to be present on the occasion, I offered t

fcm E ¥ gNlGP «® WOTH ) THE R&PP ^ Ol SPBRBTTS . [ The following Letter was written at our request . The writer , a German friend , whose integrity and clear-headedness would command attention to whatever he might assert , informed us of his proposed visit to a house where the Rappitcs were to exhibit , and we begged him to furnish us with a report . We print his letter without any alteration : its statements will , we think , set the reader speculating .Let it be distinctly borne in mind , however , that in printing this Letter we give no countenance whatever to the pretended explanation of " Rapping . " We have our own views of the jugglery by which the "ghosts " are made to communicate with persons willing to pay for the interview , and in a very short time we hope to lay before our readers something like an experimental result . ] My dear Sir , —When I proposed to give you a report on these new prophets , who , if as true as they arc new , open to us a wide and most interesting field for inquiry , overthrow ancient and modem systems of science and belief , shake to the very foundation revealed religion andChristianity , hut , on the other hand , arc telling almost equally strong against Pantheism mid Atheism , I had not seen those prophets , expounders , mediums , or whatever you may call them , myself ; hut what I heard from a friend , a clear-sighted , well-informed , by no moans " gullible , " or over-credulous gentleman , who had paid them ii visit , had made me anxious to see and judge for myself ; and he having determined , for better satisfaction , to have the Medium ( and the spirits ) at his own house , and having kindly invited me to be present on the occasion , 1 offered to furnish you with a statement of the result of the evening , which I now , agreeably to your wish , lay before you .We were five of us in the library , —my friend , his wife , his sister , his nephew , and myself , —when the footman , handing in a card , announced "Mrs . JIayden . " Her entrance and deportment were easy , unembarrassed , and yet not " business-like ; " her exterior rather prepossessing ; an intelligent countenance , with , perhaps , a slight touch of Yankccisin in the corner of the eye ; and the conversation soon bring established , showed that she did not lack those powers of speech so peculiar to the . citizens of the great Republic ; though certain mistakes now and then recurring in the conjugation of verbs , indicated that nhe could not have had a very fin ;! -rate education . We took our seats round the table on which the card had been placed , [ mid , — "Mr . and Mrs . W . R . Jlirydeii , L' 2 , Queen Anne-street , Cavenuish-. squarc . " Expectation created silence , now and then broken by . qiiestions in a low voice , addressed to the Medium , and by the wheels of carriages that brought guests to an evening party at a neighbouring house , and thus tnadd the otherwise very quirt sired ; liither noisy . Tho raps which the Medium and one or two of us , after we had been seated for about ten

Please recover the text from the corrupted OCR, providing appropriate paragraph breaks. Do not add any text, commentary, or lead in sentences  beyond the recovered text, for example for the text

"Md BRO5n0n D1e] t@ ay"

The correct response would be

"Mr Bronson Died today"

An incorrect response would be 

"Sure here is the recovered text: Mr Bronson died today"

Another incorrect response would be

"Here is the recovered text from the corrupted OCR: Mr Bronson died today"

In both cases a lead in sentence was generated which is why the example responses were incorrect

